# This notebook carries out the machine learning algorithm on the Cuebiq data. 


## Key inputs: 
* **Indicators_OurTable.csv** - features for each trip, that will be used for ML classification. 
* Random Forest Classifier trained on AIS data: **rf_model_AIS_2019.pkl**

## Key outputs:
* Indicators file with columns indicating the predicted vessel class: **Indicators_OurTable.Predictions.csv**

# Preliminaries

In [ ]:
!pip install statsmodels

In [ ]:
import pandas as pd
import os
import sys
import time
import csv
from datetime import datetime, timedelta
from datetime import datetime
import pytz  
import warnings
import numpy as np
import math
local_timezone = pytz.timezone('US/Central')

import csv
import datetime
from datetime import datetime, timedelta
from datetime import datetime
import datetime as dt
import geopandas as gpd
import heapq
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytz
import requests
import seaborn as sns
import time
import math
import statsmodels

from scipy import interpolate
from shapely import wkt
from shapely.geometry import Polygon
from zipfile import ZipFile
from shapely.geometry import Point  # Import the Point class from shapely.geometry

## Set Directories to be used

In [ ]:
#################################################################
# Directory for this output
OurTable_V3_directory = '~/RecFishing/Analysis with Our Tables and V3/Data Files'
# Expand the tilde to the user's home directory
OurTable_V3_directory = os.path.expanduser(OurTable_V3_directory)
# Check to make sure the directory exist
DirExist = os.path.exists(OurTable_V3_directory)
print(OurTable_V3_directory, "exists = " ,DirExist)


#################################################################
# Directory for output from the first draw
Batch01_directory = '~/RecFishing/Analysis with Our Tables and V3/Data Files/Batch01'
# Expand the tilde to the user's home directory
Batch01_directory = os.path.expanduser(Batch01_directory)
# Check to make sure the directory exist
DirExist = os.path.exists(Batch01_directory)
print(Batch01_directory, "exists = " ,DirExist)



#################################################################
# Directory for Groups of V3 Pings
V3_Pings_Groups_directory = '~/RecFishing/Analysis with Our Tables and V3/Data Files/V3_Ping_Groups'
V3_Pings_Groups_directory = os.path.expanduser(V3_Pings_Groups_directory)
print(V3_Pings_Groups_directory, "exists = " ,os.path.exists(V3_Pings_Groups_directory))

#################################################################
# Directory some core data fro analysis
CoreData_Directory = '~/RecFishing/CoreData'
CoreData_Directory = os.path.expanduser(CoreData_Directory)
print(CoreData_Directory, "exists = ", os.path.exists(CoreData_Directory))

#################################################################
# Directory  with Original_directory material
Original_directory = '~/RecFishing/DataflowStudioJobs'
Original_directory = os.path.expanduser(Original_directory)
DirExist = os.path.exists(Original_directory)
print(Original_directory, "exists = ", DirExist)


#################################################################
# Directory  with Original_directory material
Previously_Processed_directory = '~/RecFishing/DataflowStudioJobs/FinalCode - Rec Fishing Identification'
Previously_Processed_directory = os.path.expanduser(Previously_Processed_directory)
DirExist = os.path.exists(Previously_Processed_directory)
print(Previously_Processed_directory, "exists = ", DirExist)

#################################################################
# Directory with Travel Cost files
Travel_Cost_directory = '~/RecFishing/Travel Costs with Dedicated Table/CSV Files'
# Expand the tilde to the user's home directory
Travel_Cost_directory = os.path.expanduser(Travel_Cost_directory)
DirExist = os.path.exists(Travel_Cost_directory)
print(Travel_Cost_directory, "exists = ", DirExist)

#################################################################
# Directory with Weather data and related files
Weather_Data_directory = '~/RecFishing/uploaded_files/Weather Data'
# Expand the tilde to the user's home directory
Weather_Data_directory = os.path.expanduser(Weather_Data_directory)
print(Weather_Data_directory, "exists = ", DirExist)


#################################################################
# Directory with other Uploaded data 
Uploaded_Data_directory = '~/RecFishing/uploaded_files'
# Expand the tilde to the user's home directory
Uploaded_Data_directory = os.path.expanduser(Uploaded_Data_directory)

#################################################################
####################   Results and Analysis #################################################
Results_directory = '~/RecFishing/Analysis with Our Tables and V3/Results'
# Expand the tilde to the user's home directory
Results_directory = os.path.expanduser(Results_directory)

####################################################################################
####################  AIS Directory #################################################
AIS_Directory = '~/RecFishing/AIS Files/Data'
AIS_Directory = os.path.expanduser(AIS_Directory)
DirExist = os.path.exists(AIS_Directory)
print(AIS_Directory, "exists = ", DirExist)

# ID_list_RandomSample from ScheduledExecution5.pkl


# Files to be used

In [ ]:
def check_file_existence(file_path):
    if not os.path.exists(file_path):
        print(f"{file_path} Does NOT Exist")


######################################################################################################################
#########################  Log File  ################
Log_filename  =  os.path.join(OurTable_V3_directory, 'Log.txt')

######################################################################################################################
########################## Complete list of randomized IDs- without bernouli sampling 740k #########################
# PKL_File_With_Random_IDs_Filename  =  os.path.join(Original_directory, 'cuebiq_id_list_wo_sampling_740k.pkl')
PKL_File_With_Random_IDs_Filename  =  os.path.join(CoreData_Directory, 'cuebiq_id_list_wo_sampling_740k.pkl')
check_file_existence(PKL_File_With_Random_IDs_Filename)
    
# Data gathered and used prior to the NOAA Webinar in February 2024
IDs_Used_in_NOAA_Webinar_filename = os.path.join(CoreData_Directory, 'IDs_From_Random_Draw_Prior_to_NOAA_Webinar.csv')
check_file_existence(IDs_Used_in_NOAA_Webinar_filename)
Ping_Used_in_NOAA_Webinar_filename = os.path.join(CoreData_Directory, 'Pings_From_Random_Draw_Prior_to_NOAA_Webinar.csv')
check_file_existence(Ping_Used_in_NOAA_Webinar_filename)
                                                     
# List of IDs that have been processed for Indicators
AlreadyFullyProcessedIDs_Filename  =  os.path.join(OurTable_V3_directory, 'RandomlyChosenCuebiq_ids.List_of_Processed_ids.csv')
check_file_existence(AlreadyFullyProcessedIDs_Filename)
    
######################################################################################################################
#########################  ID Checklist with columns for ID, Pings, Indicators Created (TF) & Trips  ################
IDs_Pulled_from_Dedicated_Table_filename  =  os.path.join(OurTable_V3_directory, 'IDs_Pulled_From_Dedicated_Table.csv')
check_file_existence(IDs_Pulled_from_Dedicated_Table_filename)
    
ID_For_V3_Queries_filename  =  os.path.join(OurTable_V3_directory, 'IDs_from_V3.csv')
check_file_existence(ID_For_V3_Queries_filename)
    
RecTripRating_filename =  os.path.join(OurTable_V3_directory, 'RecTripRating.csv')
check_file_existence(RecTripRating_filename)
    
# This file contains information about the rows of Pings_V3_temp_filename that can be used to avoid loading the entire file into a data frame
V3_Pings_Index_filename =  os.path.join(OurTable_V3_directory, 'V3_Pings_File_Index.csv')
check_file_existence(V3_Pings_Index_filename)
    
ID_Groups_filename = os.path.join(OurTable_V3_directory,'Cuebiq_ID_Groups.csv')
check_file_existence(ID_Groups_filename)

######################################################
# Pings in the OurTable for a single large draw of IDs TEMPORARY FILE
Pings_OurTable_temp_filename = os.path.join(OurTable_V3_directory,'Pings_OurTable_temp.csv')
check_file_existence(Pings_OurTable_temp_filename)
    
# Pings from V3 corresponding with the IDs found in the OurTable 
Pings_V3_temp_filename = os.path.join(OurTable_V3_directory,'Pings_V3_temp.csv')
check_file_existence(Pings_V3_temp_filename)
    
# Set output file names
Indicators_IDs_checked_filename = os.path.join(OurTable_V3_directory, 'IDs_Checked_Indicators_OurTable.csv')
check_file_existence(Indicators_IDs_checked_filename)

cuebiq_id_list_and_count_filename= os.path.join(OurTable_V3_directory,'cuebiq_id_list_and_count.csv')
check_file_existence(cuebiq_id_list_and_count_filename)

# List of IDs and dates for V3 query Pings in the OurTable for a single large draw of IDs TEMPORARY FILE
OurTable_IDs_and_Dates_filename = os.path.join(OurTable_V3_directory,'OurTable_IDs_and_Dates.csv')
check_file_existence(OurTable_IDs_and_Dates_filename)
    
##########################################################################################
############################### PINGS FILES   ##############################################
Pings_OurTable_Gulf_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Gulf_ALL.csv')
check_file_existence(Pings_OurTable_Gulf_filename)

Combined_Pings_OurTable_Gulf_filename= os.path.join(OurTable_V3_directory,'Combined_Pings_OurTable_Gulf_ALL.csv')
check_file_existence(Combined_Pings_OurTable_Gulf_filename)

Pings_V3_Before_After_filename= os.path.join(OurTable_V3_directory,'Pings_V3_Before_After.csv')
check_file_existence(Pings_V3_Before_After_filename)

Combined_Pings_V3_Before_After_filename= os.path.join(OurTable_V3_directory,'Combined_Pings_V3_Before_After.csv')
check_file_existence(Combined_Pings_V3_Before_After_filename)

Pings_OurTable_Gulf_MT19_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Gulf_MT19.csv')
check_file_existence(Pings_OurTable_Gulf_MT19_filename)

Pings_OurTable_Coast_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Coast.csv')
check_file_existence(Pings_OurTable_Coast_filename)

Pings_OurTable_Outside_our_wkts_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Outside_our_wkts.csv')
check_file_existence(Pings_OurTable_Outside_our_wkts_filename)
    
##########################################################################################
############################### INDICATORS  ##############################################
Indicators_filename = os.path.join(OurTable_V3_directory,'Indicators_OurTable.csv')
check_file_existence(Indicators_filename)
    
# cuebiq_id_count_filename= os.path.join(EEZ_V3_directory,'cuebiq_id_count_distribution_EEZ_V3.csv')
Indicators_Classified_filename = os.path.join(OurTable_V3_directory,'Indicators_OurTable.Predictions.csv')
check_file_existence(Indicators_Classified_filename)

Rec_Indicators_filename = os.path.join(OurTable_V3_directory,'Rec_Indicators_OurTable.csv')
check_file_existence(Rec_Indicators_filename)

Rec_Indicators_Step1_filename = os.path.join(OurTable_V3_directory,'Rec_Indicators_OurTable.Step1.csv')
check_file_existence(Rec_Indicators_filename)

V3_Indicators_filename =  os.path.join(OurTable_V3_directory,'V3_indicators.csv')
check_file_existence(V3_Indicators_filename)

Rec_indicators_with_V3_filename = os.path.join(OurTable_V3_directory,'rec_indicators_with_V3.csv')
check_file_existence(Rec_indicators_with_V3_filename)

Indicators_with_V3_indicators_filename= os.path.join(OurTable_V3_directory,'Indicators_with_V3_indicators_indicators.csv')
check_file_existence(Indicators_with_V3_indicators_filename)

# Rec_Indicators_Selected_filename = os.path.join(OurTable_V3_directory,'Rec_Indicators_OurTable_Selected.csv')
Rec_Indicators_Selected_filename = os.path.join(OurTable_V3_directory,'Rec_Indicators_OurTable_Selected_May2024.csv')
check_file_existence(Rec_Indicators_Selected_filename)

Rec_Indicators_Final_All_Exclusions_And_Disappearance_filename = os.path.join(OurTable_V3_directory,'Rec_Indicators_Final_All_Exclusions_And_Disappearance.csv')
check_file_existence(Rec_Indicators_Final_All_Exclusions_And_Disappearance_filename)

# Sorted_Results_file_path = os.path.join(OurTable_V3_directory,'Indicators_EEZ_and_V3.Predictions.sorted.csv')
# RecFishing_Results_file_path =  os.path.join(OurTable_V3_directory,'RecFishingBoat Predictions.sorted.csv')

DisappearanceIndicators_filename = os.path.join(OurTable_V3_directory,'DisappearanceIndicators.csv')
check_file_existence(DisappearanceIndicators_filename)

DisappearanceAnalysis_filename = os.path.join(OurTable_V3_directory,'DisappearanceAnalysis.csv')
check_file_existence(DisappearanceAnalysis_filename)

Stops_Indicators_filename = os.path.join(OurTable_V3_directory,'Stops_Indicators.csv')
check_file_existence(Stops_Indicators_filename)

Trawls_Indicators_filename = os.path.join(OurTable_V3_directory,'Trawls_Indicators.csv')
check_file_existence(Trawls_Indicators_filename)

Stop_Trawls_Indicators_filename = os.path.join(OurTable_V3_directory,'Stop_Trawls_Indicators.csv')
check_file_existence(Stop_Trawls_Indicators_filename)

##########################################################################################
############################### Files that COMBINE BATCH01 and newer data ################
Combined_indicators_with_disappearance_filename = os.path.join(OurTable_V3_directory,'Combined_indicators_with_disappearance.csv')
check_file_existence(Combined_indicators_with_disappearance_filename)

Combined_Raw_indicators_filename = os.path.join(OurTable_V3_directory,'Combined_Raw_indicators.csv')
check_file_existence(Combined_Raw_indicators_filename)


## Combined file that includes the Batch01 files
Combined_Stops_Indicators_filename = os.path.join(OurTable_V3_directory,'Combined_Stops_Indicators.csv')
check_file_existence(Combined_Stops_Indicators_filename)

Combined_Trawls_Indicators_filename = os.path.join(OurTable_V3_directory,'Combined_Trawls_Indicators.csv')
check_file_existence(Combined_Trawls_Indicators_filename)

Combined_Stop_Trawls_Indicators_filename = os.path.join(OurTable_V3_directory,'Combined_Stop_Trawls_Indicators.csv')
check_file_existence(Combined_Stop_Trawls_Indicators_filename)



#################################################################
####################   Results and Analysis #################################################
Station_NonStationAnalysis_filename  = os.path.join(Results_directory,'Station_NonStationAnalysis.csv')
check_file_existence(Station_NonStationAnalysis_filename)

Station_NonStationAnalysis_full_filename  = os.path.join(Results_directory,'Station_NonStationAnalysis_full.csv')
check_file_existence(Station_NonStationAnalysis_full_filename)


##########################################################################################
############################### WEATHER data files ####################################
Buoys_file_path  = os.path.join(Weather_Data_directory,'Buoys.csv')
check_file_existence(Buoys_file_path)

Weather_file_path  = os.path.join(Weather_Data_directory,'DailyWeatherData.csv')
check_file_existence(Weather_file_path)

##########################################################################################
############################### SUPPLEMENTARY MAP DATA  ############################
Industrial_polygons_filename  = os.path.join(Uploaded_Data_directory,'Polygons Around Industrial Sites.wkt')
check_file_existence(Industrial_polygons_filename)


station_points_filename = os.path.join(Uploaded_Data_directory,'LA_TX_Union_Station_WGS84.csv')
check_file_existence(station_points_filename)

MRIP_station_points_filename =os.path.join(Uploaded_Data_directory,'MRIP_stations.csv')
check_file_existence(MRIP_station_points_filename)

##########################################################################################
############################### AIS Files INCLUDING CLASSIFIER ############################
RF_Classfier_filename = os.path.join(AIS_Directory, 'rf_model_AIS_2019.pkl')
check_file_existence(RF_Classfier_filename)

RF_Importance_Factors_filename = os.path.join(AIS_Directory, 'rf_classifier_importance_factors.csv')
check_file_existence(RF_Importance_Factors_filename)

AIS_Predictions_filename = os.path.join(AIS_Directory, 'RandomForest_Predictions2019AISData.csv')
check_file_existence(AIS_Predictions_filename)
    
AIS_indicators_file_path = os.path.join(AIS_Directory,'Indicators2019_All.C.csv')
check_file_existence(AIS_indicators_file_path)
  

### Dedicate Table Names for reference
# Dedicated table with all Pings within the Gulf WKT for 1/12019 - 4/22/2022
#  Table Name:  dedicated.ScheduledExecution5.DeviceTable   
#  Code used for call:  RecFishing/DataflowStudioJobs/ScheduledEx5-updated.ipynb

# Dedicated table with all Pings within the Gulf WKT AND Origin for 1/12019 - 4/22/2022
#  Table Name:  dedicated.ScheduledExecution5_parallel_origin.DeviceTable
#  Code used for call:  RecFishing/DataflowStudioJobs/ScheduledEx5-origin.ipynb

##########################################################################################
############################### Results & Analysis ############################


# Correct the Indicators for the problem in the calculation of trip duration

In [ ]:
########### This Function works on an entire data frame ##################
##  Use
## df = df_duration_correction(df)
def df_duration_correction(df):
    # Calculate corrected trip duration
    df['Trip_Duration_t_Corrected'] = (df['timestamp_end_t'] - df['timestamp_start_t']) / 60
    
    # Calculate duration correction factor and apply to other columns
    df['Duration_Correction'] = df['Trip_Duration_t'] / df['Trip_Duration_t_Corrected']
    df['Pings_per_minute_t'] = df['Pings_per_minute_t'] * df['Duration_Correction']
    df['pct_time_stopped'] = df['pct_time_stopped'] * df['Duration_Correction']
    df['pct_time_trawling'] = df['pct_time_trawling'] * df['Duration_Correction']
    df['pct_time_moving'] = df['pct_time_moving'] * df['Duration_Correction']
    
    # If Trip_Duration_t_Corrected is zero, set all related columns to zero
    zero_duration_mask = df['Trip_Duration_t_Corrected'] == 0
    df.loc[zero_duration_mask, ['Pings_per_minute_t', 'pct_time_stopped', 'pct_time_trawling', 'pct_time_moving']] = 0
    
    # Update Trip_Duration_t to corrected value
    df['Trip_Duration_t'] = df['Trip_Duration_t_Corrected']
    
    # Drop intermediate columns
    df.drop(columns=['Trip_Duration_t_Corrected', 'Duration_Correction'], inplace=True)
    
    return df

########### This Function only works on one value at a time ##################
def duration_correction(timestamp_end_t, timestamp_start_t, Trip_Duration_t, 
                        Pings_per_minute_t, pct_time_stopped, pct_time_trawling, 
                        pct_time_moving):

    Trip_Duration_t_Corrected = (timestamp_end_t - timestamp_start_t) / 60
    
    print("Trip_Duration_t, Trip_Duration_t_Corrected", Trip_Duration_t, Trip_Duration_t_Corrected)
    if Trip_Duration_t_Corrected > 0:
        Duration_Correction = Trip_Duration_t / (Trip_Duration_t_Corrected)

        Pings_per_minute_t = Pings_per_minute_t * Duration_Correction
        pct_time_stopped = pct_time_stopped * Duration_Correction
        pct_time_trawling = pct_time_trawling * Duration_Correction
        pct_time_moving = pct_time_moving * Duration_Correction
    else:
        Duration_Correction = 0

        Pings_per_minute_t = 0
        pct_time_stopped = 0
        pct_time_trawling = 0
        pct_time_moving = 0

    Trip_Duration_t =Trip_Duration_t_Corrected
    return Trip_Duration_t, Pings_per_minute_t, pct_time_stopped, pct_time_trawling, pct_time_moving


# Use the RF classifier to categorize the observations from the Spectus Data

In [ ]:
import joblib
# Load the trained Random Forest classifier from the saved file
rf_classifier = joblib.load(RF_Classfier_filename)

# Load the new dataset
Complete_Indicators_df = pd.read_csv(Combined_Raw_indicators_filename)

# # Correct the durations, which had problems in the original indicators file
# Complete_Indicators_df = df_duration_correction(Complete_Indicators_df)


In [ ]:
Combined_Raw_indicators_filename

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import datetime

# The set of variables used to predict must match those used in the original random forest classification 
X = Complete_Indicators_df[[
                        'trips_per_day', 
                        'break_duration_t',
                        'Weekday_trips', 
                        'Trip_Duration_t',
                        'Total_distance_traveled_t', 
                        'Distance_from_origin_t', 
                        'first_distance_from_coast_t',
                        'last_distance_from_coast_t', 
                        'Max_distance_traveled_origin_t',
                        'Weekend_trip_t', 
                        'time_stopped_t', 
                        'pct_time_stopped',
                        'number_of_stops_t', 
                        'longest_stop_t',
                        'shortest_stop_t',
                        'time_trawling_t',
                        'pct_time_trawling',
                        'number_of_trawl_t',
                        'longest_trawl_t', 
                        'shortest_trawl_t', 
                        'distance_trawling_t',
                        'time_moving_t', 
                        'pct_time_moving', 
                        'number_of_moves_t',
                        'longest_move_t', 
                        'shortest_move_t',
                        'distance_moving_t',
                        'dist_from_origin_during_stops', 
                        'Trip_pings_mov_traw_per_time_t',
                        'move_efficiency_t', 
                        'move_speed_t',
                        'maxspeed_t', 
                        'max_distance_from_coast_t',
                        'WSPD', 
                        'GST',
                        'WVHT', 
                        'ATMP'   
                        ]]


# Assuming you have preprocessed your training data, preprocess the new dataset similarly
# Use the trained rf_classifier to make predictions on the new dataset
predictions = rf_classifier.predict(X)

# Use the trained rf_classifier to predict class probabilities
class_probabilities = rf_classifier.predict_proba(X)

# Get the class labels
class_labels = rf_classifier.classes_

# Combine the predictions, class probabilities, and original features
Indicators_with_RF_Predictions_df = pd.DataFrame({
    'Predicted_Class': rf_classifier.predict(X),
    **{f'Probability_{class_labels[i]}': class_probabilities[:, i] for i in range(len(class_labels))},
    **Complete_Indicators_df  # Include original features from the new dataset
})

####################### Clean up and add max_prob column #############################
# Add a new column 'max_prob' with the maximum probability from specified columns
Indicators_with_RF_Predictions_df['max_prob'] = Indicators_with_RF_Predictions_df[['Probability_30', 'Probability_31', 'Probability_60',
                                             'Probability_70', 'Probability_80', 'Probability_90',
                                             'Probability_371', 'Probability_372']].max(axis=1)
# Sort the DataFrame based on 'max_prob' in descending order
Indicators_with_RF_Predictions_df = Indicators_with_RF_Predictions_df.sort_values(by='max_prob', ascending=False)

# Reorder the columns as specified
column_order = ['cuebiq_id', 'Trip_number','Predicted_Class', 'max_prob','Probability_30', 'Probability_31', 'Probability_60',
                                             'Probability_70', 'Probability_80', 'Probability_90',
                                             'Probability_371', 'Probability_372'] # sorted_feature_names  # 'feature_names' contains the ordered feature names
other_columns = [col for col in Indicators_with_RF_Predictions_df.columns if col not in column_order]
new_column_order = column_order + other_columns
Indicators_with_RF_Predictions_df = Indicators_with_RF_Predictions_df[new_column_order]


######################## Save the combined data to a CSV file  #######################
Indicators_with_RF_Predictions_df.to_csv(Indicators_Classified_filename, index=False)

classes_to_count = [371, 30, 31, 60, 70, 80, 90, 372]

# Filter the DataFrame for each class and count the number of rows
for class_num in classes_to_count:
    count = Indicators_with_RF_Predictions_df[Indicators_with_RF_Predictions_df['Predicted_Class'] == class_num].shape[0]
    percent = round(100*count/len(Indicators_with_RF_Predictions_df),0)
    print(f"Predictions of {class_num}: {count}  {percent}%")

print(f"Total:         {len(Indicators_with_RF_Predictions_df)}")


In [ ]:
# Store sorted results into a csv file

RecFishing_Predictions = Indicators_with_RF_Predictions_df[(Indicators_with_RF_Predictions_df['Predicted_Class'] == 371) & (Indicators_with_RF_Predictions_df['max_prob'] >= 0.5)].copy()
Indicators_with_RF_Predictions_df.to_csv(Sorted_Results_file_path, index=False)
RecFishing_Predictions.to_csv(REcFishing_Results_file_path, index=False)


current_time = datetime.now()
formatted_time = current_time.strftime("%H:%M:%S")

print(Sorted_Results_file_path, " and ", REcFishing_Results_file_path, "created at  ",  formatted_time)

# 4. Analysis with categorized Spectus data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# File path and DataFrame loading
RF_predictions_sorted = pd.read_csv(Sorted_Results_file_path)

# Filter the DataFrame based on the conditions
RFB_df = RF_predictions_sorted[(RF_predictions_sorted['Predicted_Class'] == 371) & (RF_predictions_sorted['max_prob'] >= 0.1)]

# Initialize lists to store results
thresholds = []
count_rows_list = []
unique_ids_list = []

# Iterate over the probability thresholds (0.1, 0.2, ..., 0.9)
for threshold in range(1, 10):
    threshold /= 10.0  # Convert to decimal form
    threshold_condition = RFB_df['max_prob'] >= threshold
    count_rows = len(RFB_df[threshold_condition])
    unique_ids = RFB_df[threshold_condition]['id'].nunique()
    thresholds.append(threshold)
    count_rows_list.append(count_rows)
    unique_ids_list.append(unique_ids)

# Display the results
print("Recreational fishing boats")
for threshold, count_rows, unique_ids in zip(thresholds, count_rows_list, unique_ids_list):
    print(f"For max_prob >= {threshold:.1f}:", f"  Number of trips: {count_rows}", f"  Number of ids: {unique_ids}")

# Create a line graph
plt.plot(thresholds, count_rows_list, label='Number of trips')
plt.plot(thresholds, unique_ids_list, label='Number of devices')
plt.xlabel('Minimum probability for inclusion')
plt.ylabel('Count')
# plt.title('Number of trips and unique ids by max_prob threshold')

# Format the horizontal axis as percentages
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:.0%}'.format(x)))

plt.legend()
plt.grid(True)
plt.show()
